In [13]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [4]:
img = cv2.imread('img.jpeg')


In [5]:
gray_picture = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#make picture gray
gray_picture

array([[197, 197, 197, ..., 203, 203, 203],
       [197, 197, 197, ..., 203, 203, 203],
       [197, 197, 197, ..., 203, 203, 203],
       ...,
       [ 58,  53,  47, ...,  65,  65,  63],
       [ 58,  53,  47, ...,  65,  65,  63],
       [ 58,  53,  47, ...,  65,  65,  63]], dtype=uint8)

In [6]:
faces = face_cascade.detectMultiScale(gray_picture, 1.3, 5)

faces

array([[42, 54, 92, 92]], dtype=int32)

In [7]:
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)

In [8]:
cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
gray_face = gray_picture[y:y+h, x:x+w] # cut the gray face frame out
face = img[y:y+h, x:x+w] # cut the face frame out
eyes = eye_cascade.detectMultiScale(gray_face)

In [10]:
for (ex,ey,ew,eh) in eyes: 
    cv2.rectangle(face,(ex,ey),(ex+ew,ey+eh),(0,225,255),2)

In [11]:
cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
def detect_eyes(img, classifier):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = eyecenter.detectMultiScale(gray_frame, 1.3, 5) # detect eyes
    width = np.size(img, 1) # get face frame width
    height = np.size(img, 0) # get face frame height
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye center
        if eyecenter < width * 0.5:
            left_eye = img[y:y + h, x:x + w]
        else:
            right_eye = img[y:y + h, x:x + w]
    return left_eye, right_eye

In [16]:
def detect_faces(img, classifier):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return None
    for (x, y, w, h) in biggest:
        frame = img[y:y + h, x:x + w]
    return frame

In [17]:
detector_params = cv2.SimpleBlobDetector_Params()
detector_params.filterByArea = True
detector_params.maxArea = 1500
detector = cv2.SimpleBlobDetector_create(detector_params)

In [18]:
img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)

In [20]:
def cut_eyebrows(img):
    height, width = img.shape[:2]
    eyebrow_h = int(height / 4)
    img = img[eyebrow_h:height, 0:width]  # cut eyebrows out (15 px)
    return img

In [21]:
def blob_process(img, detector):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(gray_frame, 42, 255, cv2.THRESH_BINARY)
    keypoints = detector.detect(img)
    return keypoints
keypoints = blob_process(eye, detector)
cv2.drawKeypoints(eye, keypoints, eye, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

NameError: name 'eye' is not defined

In [ ]:
import cv2
import numpy as np
import pyautogui
import time

# Load cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

if face_cascade.empty() or eye_cascade.empty():
    print("Error loading cascade files.")
    exit()

# Webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot access camera.")
    exit()

scroll_cooldown = 1.0  # seconds
last_scroll_time = time.time()

# Threshold to consider eye closed
INTENSITY_THRESHOLD = 60

def is_eye_closed(eye_gray):
    # Get average brightness of eye region
    brightness = np.mean(eye_gray)
    return brightness < INTENSITY_THRESHOLD

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_gray  = gray[y:y+h, x:x+w]
        face_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(face_gray, 1.3, 5)
        eyes = sorted(eyes, key=lambda e: e[0])  # Sort by x to get left/right order

        if len(eyes) >= 2:
            (lx, ly, lw, lh) = eyes[0]  # Left eye
            (rx, ry, rw, rh) = eyes[1]  # Right eye

            left_eye  = face_gray[ly:ly+lh, lx:lx+lw]
            right_eye = face_gray[ry:ry+rh, rx:rx+rw]

            now = time.time()

            if is_eye_closed(left_eye) and not is_eye_closed(right_eye):
                if now - last_scroll_time > scroll_cooldown:
                    pyautogui.scroll(-150)  # Scroll down
                    print("↓ Left eye closed → Scroll down")
                    last_scroll_time = now

            elif is_eye_closed(right_eye) and not is_eye_closed(left_eye):
                if now - last_scroll_time > scroll_cooldown:
                    pyautogui.scroll(150)  # Scroll up
                    print("↑ Right eye closed → Scroll up")
                    last_scroll_time = now

            # Optional: visualize eyes
            cv2.rectangle(face_color, (lx, ly), (lx+lw, ly+lh), (255, 0, 0), 2)
            cv2.rectangle(face_color, (rx, ry), (rx+rw, ry+rh), (0, 255, 0), 2)

    cv2.imshow("Eye Blink Scroll", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Baseline set at: 19.50
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
↓ Scroll Down
